In [1]:
import os
import numpy as np
import pandas as pd

c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Get the images

In [2]:
dir_path    = 'C:\\Users\\nperc\\Documents\\Datathon\\data\\data_image\\data_3pic\\'

dataset = pd.read_csv('C:\\Users\\nperc\\Documents\\Datathon\\clean_extract_model\\data\\sheet\\data_final_lite.csv')
dataset = dataset.loc[:, ['ID', 'CHA2DS2-VASc', 'Age', 'Sexe']]

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nperc\\Documents\\Datathon\\clean_deep_model\\data\\sheet\\data_final_lite.csv'

In [ ]:
SIZE = 224

image_size  = (SIZE, SIZE, 3)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model

efficient_model = EfficientNetB0(weights=None)

model = Model(inputs=efficient_model.inputs, outputs=efficient_model.layers[-2].output)

model.summary()

In [ ]:
import cv2 as cv
from tensorflow.keras.preprocessing.image import load_img, img_to_array

error_images       = []
patient_id_list    = []

def find_images(name) :
    data        = []
    

    for dir_client in os.listdir(dir_path) :
        path_client = os.path.join(dir_path, dir_client)
        for dirs in os.listdir(path_client) :
            if dirs.lower() == name + ' od.bmp' or dirs.lower() == name + ' og.bmp' :
                image = cv.imread(os.path.join(path_client, dirs))
                image = cv.resize(image, (SIZE, SIZE), interpolation = cv.INTER_LINEAR)
                
                # image[image > 235]  -= 20
                # image[image > 50]   += 20
                
                if(image[image.astype(bool)].size != 0) :
                    try :
                        # data = image_augmentation(data, image, model, dir_client)
                        age     = round(dataset['Age'][dataset['ID'] == dir_client].values[0])
                        Sexe  = int(dataset['Sexe'][dataset['ID'] == dir_client].values[0])
                        target  = dataset['CHA2DS2-VASc'][dataset['ID'] == dir_client].values[0]
                        patient_id_list.append(dir_client)
                        data.append([image, age, Sexe, target])
                    except :
                        if dir_client not in error_images :
                            error_images.append(dir_client)
                        continue
    return data

In [ ]:
columns = ['image', 'age', 'Sexe', 'target']

cc_images     = pd.DataFrame(find_images('cc'), columns=columns)
sup_images    = pd.DataFrame(find_images('sup'), columns=columns)
deep_images   = pd.DataFrame(find_images('deep'), columns=columns)

In [ ]:
def form_col(data) :
    columns = []
    for col in data.columns.to_list() :
        columns.append(str(col))
        
    new_columns = columns[:-3]
    new_columns.append('age')
    new_columns.append('Sexe')
    new_columns.append('target')
        
    data.columns = new_columns
    return data
    
cc_images     = form_col(cc_images)
sup_images    = form_col(sup_images)
deep_images   = form_col(deep_images)

In [6]:
cc_data     = pd.read_csv('C:\\Users\\nperc\\Documents\\Datathon\\clean_deep_model\\data\\cc_EfficientNetB0_ageSexe_new.csv')
sup_data    = pd.read_csv('C:\\Users\\nperc\\Documents\\Datathon\\clean_deep_model\\data\\sup_EfficientNetB0_ageSexe_new.csv')
deep_data   = pd.read_csv('C:\\Users\\nperc\\Documents\\Datathon\\clean_deep_model\\data\\deep_EfficientNetB0_ageSexe_new.csv')

In [7]:
age         = cc_data['age']
sexe        = cc_data['Sexe']
target      = cc_data['target']

cc_data     = cc_data.iloc[:, :-1]
sup_data    = sup_data.iloc[:, :-1]
deep_data   = deep_data.iloc[:, :-1]

## PCA

In [277]:
def find_nb_pca(data, numerical_columns) :
    data_sample = data.loc[:, numerical_columns]

    data_sum = data_sample.sum().sort_values(axis=0, ascending = False)
    sum = 0
    for value in data_sum :
        sum += value
        
    value_perc = []

    sum_perc = 0
    for i in range(len(data_sum)) :
        
        perc = data_sum[i] / sum
        value_perc.append(perc)
        sum_perc += perc
        if sum_perc >= 0.95 :
            break
    return i + 1

# Feature Extraction + Classification per depth

In [278]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


def make_preprocessor(data, numeric_columns):
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=find_nb_pca(data, numeric_columns))),    
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_columns),
        ])
    return preprocessor

In [279]:
def find_num_col(X, binary_columns=[], categorical_columns=[]) :
    
    numeric_columns = []
    for elem in X.columns.to_list():
        if elem not in binary_columns and elem not in categorical_columns :
            numeric_columns.append(elem)
    return numeric_columns

In [8]:
cc_y    = target
cc_X    = cc_data

sup_y   = target
sup_X   = sup_data

deep_y  = target
deep_X  = deep_data

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


def make_forest_model(X_train, y_train) :
    forest_model = RandomForestClassifier(random_state=42)

    forest_param = {
        'bootstrap'         : [True],
        'max_depth'         : [5, 8, 12, 15, 20],
        'ccp_alpha'         : [.001],
        'n_estimators'      : [25, 50, 100],
        'criterion'         : ['gini', 'entropy']
        }

    forest_grid = GridSearchCV(forest_model, forest_param)
    
    forest_grid.fit(X_train, y_train)
    print(forest_grid.best_params_)
    
    return forest_grid.best_estimator_

forest_model = RandomForestClassifier(random_state=42)

forest_param = {
    'bootstrap'         : [True],
    'max_depth'         : [5, 8, 12, 15, 20],
    'ccp_alpha'         : [.001],
    'n_estimators'      : [25, 50, 100],
    'criterion'         : ['gini', 'entropy']
    }

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def train_model(model, X_train, y_train, X_test, y_test, param=None, scoring='accuracy') :
    
    best_score      = 0
    grid_list       = []
    metric_list     = []
    estimator_list  = []
    
    dict_scoring    = {
        'accuracy'      : accuracy_score,
        'precision'     : precision_score,
        'recall'        : recall_score,
        'f1'            : f1_score
    }
    
    if param == None :
        model.fit(X_train, y_train)
        
        y_pred  = model.predict(X_test)
        score   = dict_scoring[scoring](y_test, y_pred)
        
        grid_list.append(g)
        estimator_list.append(model)
        metric_list.append([score, classification_report(y_test, y_pred), confusion_matrix(y_test, y_pred)])
        
    else :
        for g in ParameterGrid(param):
            
            model.set_params(**g)
            model.fit(X_train, y_train)
            
            y_pred  = model.predict(X_test)
            score   = dict_scoring[scoring](y_test, y_pred)
            
            estimator_list.append(model)
            metric_list.append([score, classification_report(y_test, y_pred), confusion_matrix(y_test, y_pred)])
            if score > best_score:
                best_grid       = g
                best_score      = score
                best_estimator  = model
                
            
    return best_estimator, best_grid, best_score, estimator_list, grid_list, metric_list

In [13]:
from sklearn.model_selection import KFold, train_test_split

def dataset_split_index(X, y, test_size = 0.2, fold=None, random_state=42) :
    
    if fold == None :
        data_train, data_test, labels_train, labels_test, index_train, index_test = train_test_split(X, y, X.index, test_size=test_size, random_state=random_state)
        return [index_train, index_test]
    else :
        index_list  = []
        kf          = KFold(n_splits=fold, random_state=random_state, shuffle=True)
        for i, (index_train, index_test) in enumerate(kf.split(X)) :
            index_list.append([index_train, index_test])
        return index_list

In [ ]:
from skimage import color
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

  
def form_image(image) :
    image = image / 255.
    image = np.expand_dims(image,axis=0)
    return image

def predict_image(image, model) :    
    form_data = []
    
    for elem in model.predict(image)[0] :
        form_data.append(elem)
    return form_data
  
def image_augmentation(images, model) :

    data = []
    for i in range(len(images)) :
        
        age     = round(dataset['Age'][dataset['ID'] == patient_id_list[i]].values[0])
        Sexe  = int(dataset['Sexe'][dataset['ID'] == patient_id_list[i]].values[0])
        
        data.append([predict_image(form_image(images[i]), model), age, Sexe])

        # data.append([predict_image(form_image(adjust_gamma(images[i], gamma=0.5,gain=1)), model), age, Sexe])
        # data.append([predict_image(form_image(adjust_gamma(images[i], gamma=2,gain=1)), model), age, Sexe])
        
        data.append([predict_image(form_image(np.fliplr(images[i])), model), age, Sexe])
        data.append([predict_image(form_image(np.flipud(images[i])), model), age, Sexe])
        
        # data.append([predict_image(form_image(random_noise(images[i])), model), age, Sexe])

    return data


from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0

def preprocess_image(X, y, index_train, index_test, image_augment=False, preprocess_model=None) :
    
    if preprocess_model == None :
        efficient_model     = EfficientNetB0(weights=None)
        preprocess_model    = Model(inputs=efficient_model.inputs, outputs=efficient_model.layers[-2].output)
    
    X_train     = X.iloc[index_train, :]
    y_train     = y.iloc[index_train, :]
    
    X_test      = X.iloc[index_test, :]
    y_test      = y.iloc[index_test, :]
    
    if image_augment :
        X_train = form_col(pd.DataFrame(image_augmentation(X_train['image'], preprocess_model)))
    else :
        for i in range(X_train.shape[0]) :
            X_train['image'][i] = predict_image(form_image(X_train['image'][i]), preprocess_model)
        

In [ ]:
def split_train_model(X, y, model, grid_param=None, fold=None, test_size = 0.2, random_state=42, image_augmentation=False, scoring='accuracy') :
    
    best_score      = 0    
    report_list     = []
    
    for index_train, index_test in enumerate(dataset_split_index(X, y, test_size=test_size, fold=fold, random_state=random_state)) :
        
        X_train, y_train, X_test, y_test = preprocess_image()
        estimator, grid, score, estimator_list, grid_list, metric_list = train_model(model, X_train, y_train, X_test, y_test, param=grid_param, scoring=scoring)
        
        report_list.append(estimator, grid, score, estimator_list, grid_list, metric_list)
        
        if score > best_score:
            best_score      = score
            best_estimator  = model
    return best_estimator, best_score, report_list

In [281]:
cc_preprocess   = make_preprocessor(cc_data, find_num_col(cc_data))
sup_preprocess  = make_preprocessor(sup_data, find_num_col(sup_data))
deep_preprocess = make_preprocessor(deep_data, find_num_col(deep_data))

In [282]:
cc_X            = cc_preprocess.fit_transform(cc_X)
sup_X           = sup_preprocess.fit_transform(sup_X)
deep_X          = deep_preprocess.fit_transform(deep_X)

In [283]:
from sklearn.model_selection import train_test_split

cc_X_train, cc_X_test, cc_y_train, cc_y_test            = train_test_split(cc_X, cc_y, test_size = 0.2, random_state=42)
sup_X_train, sup_X_test, sup_y_train, sup_y_test        = train_test_split(sup_X, sup_y, test_size = 0.2, random_state=42)
deep_X_train, deep_X_test, deep_y_train, deep_y_test    = train_test_split(deep_X, deep_y, test_size = 0.2, random_state=42)

In [284]:
from imblearn.over_sampling import SMOTE

# sm = SMOTE(random_state=42, k_neighbors=150)
# cc_X_train, cc_y_train = sm.fit_resample(cc_X_train, cc_y_train)

# sm = SMOTE(random_state=42, k_neighbors=150)
# sup_X_train, sup_y_train = sm.fit_resample(sup_X_train, sup_y_train)

# sm = SMOTE(random_state=42, k_neighbors=150)
# deep_X_train, deep_y_train = sm.fit_resample(deep_X_train, deep_y_train)

In [285]:
# cc_X_train    = cc_preprocess.fit_transform(cc_X_train)
# sup_X_train   = sup_preprocess.fit_transform(sup_X_train)
# deep_X_train  = deep_preprocess.fit_transform(deep_X_train)

In [287]:
import numpy as np
from sklearn.linear_model import LogisticRegression

def make_LR_model(X_train, y_train) :

    lr_model = LogisticRegression(random_state=42)

    param = {"C":np.logspace(-3,3,7), "penalty":["l2"]}

    lr_grid = GridSearchCV(lr_model, param, scoring='f1_samples')
    
    lr_grid.fit(X_train, y_train)
    print(lr_grid.best_params_)
    
    return lr_grid.best_estimator_

In [288]:
from sklearn.svm import SVC


def make_SVC_model(X_train, y_train) :
    svc = SVC(random_state=42)

    param = {
        'C'         :[0.1,1,10,100],
        'gamma'     :[1,0.1,0.001],
        'kernel'    :['linear', 'poly', 'rbf', 'sigmoid']
    }


    svc_grid = GridSearchCV(svc, param)
        
    svc_grid.fit(X_train, y_train)
    print(svc_grid.best_params_)
    
    return svc_grid.best_estimator_

In [289]:
cc_model   = make_LR_model(cc_X_train, cc_y_train)
sup_model  = make_LR_model(sup_X_train, sup_y_train)
deep_model = make_LR_model(deep_X_train, deep_y_train)

c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "c:\Users\nperc\Documents\Datathon\01_i

{'C': 0.001, 'penalty': 'l2'}


c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "c:\Users\nperc\Documents\Datathon\01_i

{'C': 0.001, 'penalty': 'l2'}
{'C': 0.001, 'penalty': 'l2'}


c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "c:\Users\nperc\Documents\Datathon\01_i

In [290]:
# cc_model   = LogisticRegression(random_state=42, C=1000, penalty='l1', solver='liblinear').fit(cc_X_train, cc_y_train)
# sup_model  = LogisticRegression(random_state=42, C=1000, penalty='l1', solver='liblinear').fit(sup_X_train, sup_y_train)
# deep_model = LogisticRegression(random_state=42, C=1000, penalty='l1', solver='liblinear').fit(deep_X_train, deep_y_train)

In [291]:
cc_y_pred   = cc_model.predict(cc_X)
sup_y_pred  = sup_model.predict(sup_X)
deep_y_pred = deep_model.predict(deep_X)

In [292]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report


all_pred = cc_model.predict(cc_X_test)

print(classification_report(cc_y_test, all_pred))
print(confusion_matrix(cc_y_test, all_pred))

c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

              precision    recall  f1-score   support

           0       0.56      0.82      0.66       202
           1       0.47      0.35      0.40       162
           2       0.00      0.00      0.00        56

    accuracy                           0.53       420
   macro avg       0.34      0.39      0.36       420
weighted avg       0.45      0.53      0.47       420

[[166  36   0]
 [105  57   0]
 [ 27  29   0]]


In [293]:
age.min()

11

In [294]:
cc_y_pred_test   = cc_model.predict(cc_X_test)
sup_y_pred_test  = sup_model.predict(sup_X_test)
deep_y_pred_test = deep_model.predict(deep_X_test)

In [295]:
y_pred = []

for i in range(len(cc_y_pred_test)) :
    if cc_y_pred_test[i] == deep_y_pred_test[i] :
        y_pred.append(cc_y_pred_test[i])
    else :
        y_pred.append(sup_y_pred_test[i])

In [296]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

print(classification_report(cc_y_test, y_pred))
print(confusion_matrix(cc_y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.85      0.65       202
           1       0.48      0.27      0.35       162
           2       0.00      0.00      0.00        56

    accuracy                           0.51       420
   macro avg       0.34      0.37      0.33       420
weighted avg       0.44      0.51      0.45       420

[[172  30   0]
 [118  44   0]
 [ 39  17   0]]


c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

# Classification with all depth + New Values

In [297]:
all_X = []

for i in range(len(cc_y_pred)) :
    all_X.append([age[i], sexe[i], cc_y_pred[i], sup_y_pred[i], deep_y_pred[i]])
    
all_X = pd.DataFrame(all_X, columns=['age','sex', 'cc_pred','sup_pred', 'deep_pred'])
all_y = target

In [298]:
from sklearn.preprocessing import OneHotEncoder

def all_make_preprocessor(numeric_columns, categorical_columns):

    numeric_transformer     = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        # ('pca', PCA(n_components=find_nb_pca(data, numeric_columns))),    
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_columns),
            ('categorical', categorical_transformer, categorical_columns),            
        ])
    return preprocessor

In [299]:
numeric_columns     = ['age']
binary_columns      = ['Sexe']
categorical_columns = [col for col in all_X.columns if col not in numeric_columns + binary_columns]

all_preprocessor    = all_make_preprocessor(numeric_columns, categorical_columns)

all_X = all_preprocessor.fit_transform(all_X)

In [300]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_X, all_y, test_size = 0.2, random_state=42)

In [301]:
all_model   = make_forest_model(X_train, y_train)

{'bootstrap': True, 'ccp_alpha': 0.001, 'criterion': 'entropy', 'max_depth': 15, 'n_estimators': 50}


In [302]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report


all_pred = all_model.predict(X_test)

print(classification_report(y_test, all_pred))
print(confusion_matrix(y_test, all_pred))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87       202
           1       0.75      0.78      0.77       162
           2       0.75      0.64      0.69        56

    accuracy                           0.80       420
   macro avg       0.79      0.76      0.77       420
weighted avg       0.80      0.80      0.80       420

[[175  25   2]
 [ 25 127  10]
 [  2  18  36]]


In [303]:
27/420

0.06428571428571428

In [305]:
# prep = cc_preprocess.transform(pd.DataFrame([[90, 1]], columns=['age', 'Sexe']))
# prep